## Imports & Tests

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os 

# np.set_printoptions(threshold=np.nan)

In [2]:
#Test tensorflow is working
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

a = tf.constant(10)
b = tf.constant(32)
print(sess.run(a + b))

b'Hello, TensorFlow!'
42


## Load in Data

In [3]:
#p3 removed
data_a_files = ["p1_a.txt","p2_a.txt","p4_a.txt","p5_a.txt","p6_a.txt","p7_a.txt","p8_a.txt","p9_a.txt","p10_a.txt","p11_a.txt","p12_a.txt","p13_a.txt"]
data_b_files = ["p1_b.txt","p2_b.txt","p4_b.txt","p5_b.txt","p6_b.txt","p7_b.txt","p8_b.txt","p9_b.txt","p10_b.txt","p11_b.txt","p12_b.txt","p13_b.txt"]
data_c_files = ["p1_c.txt","p2_c.txt","p4_c.txt","p5_c.txt","p6_c.txt","p7_c.txt","p8_c.txt","p9_c.txt","p10_c.txt","p11_c.txt","p12_c.txt","p13_c.txt"]
data_d_files = ["p1_d.txt","p2_d.txt","p4_d.txt","p5_d.txt","p6_d.txt","p7_d.txt","p8_d.txt","p9_d.txt","p10_d.txt","p11_d.txt","p12_d.txt","p13_d.txt"]

column_names = ['timestamp', 'x', 'y', 'z', 'ax', 'ay', 'az', 'gsr', 'hr', 'endline']
new_col_names = ['x', 'y', 'z', 'ax', 'ay', 'az', 'gsr', 'hr', 'avg_x', 'avg_y', 'avg_z', 'avg_ax', 'avg_ay', 'avg_az', 
                 'avg_gsr', 'avg_hr', 'x_num_max', 'x_num_min', 'y_num_max', 'y_num_min', 'z_num_max', 'z_num_min', 
                 'ax_num_max', 'ax_num_min', 'ay_num_max', 'ay_num_min', 'az_num_max', 'az_num_min',
                 'gsr_num_max', 'gsr_num_min', 'hr_num_max', 'hr_num_min',
                'max_x', 'min_x', 'diff_max_x', 'diff_min_x', 
                'max_y', 'min_y', 'diff_max_y', 'diff_min_y', 
                'max_z', 'min_z', 'diff_max_z', 'diff_min_z',
                'max_ax', 'min_ax', 'diff_max_ax', 'diff_min_ax', 
                'max_ay', 'min_ay', 'diff_max_ay', 'diff_min_ay', 
                'max_az', 'min_az', 'diff_max_az', 'diff_min_az',
                'max_gsr', 'min_gsr', 'diff_max_gsr', 'diff_min_gsr', 
                'max_hr', 'min_hr', 'diff_max_hr', 'diff_min_hr', 'thresh_gsr', 'thresh_hr']

In [4]:
###Load A and get thresholds

master_a = pd.DataFrame()
thresholds = []
i = 0

for filename in data_a_files:
    
    i = i+1
    load_data = pd.read_csv(filename, sep=",", names=column_names)
    load_data_thresholds = [i, load_data["x"].mean(), load_data["y"].mean(), load_data["z"].mean(), load_data["ax"].mean(), 
                            load_data["ay"].mean(), load_data["az"].mean(), load_data["gsr"].mean(), load_data["hr"].mean(), "~"]

    thresholds.append([load_data_thresholds])

In [5]:
# #Get average gsr and hr over all relaxed data
gsr_avg = 0.0
hr_avg = 0.0

for entry in thresholds:
    gsr_avg = gsr_avg + entry[0][7]
    hr_avg = hr_avg + entry[0][8]
    
gsr_avg = gsr_avg/len(thresholds)
hr_avg = hr_avg/len(thresholds)


In [6]:
def getMaximaMinima(np_arr):
    
    gradients=np.diff(np_arr)
    
    max_count = 0
    min_count = 0
    count = 0
    
    for i in gradients[:-1]:
        count = count + 1

        if ((i>0) & (gradients[count]<=0) & (i != gradients[count])):
            max_count+=1 

        if ((0>i) & (gradients[count]>=0) & (i != gradients[count])):
            min_count+=1

    return [max_count, min_count]

In [7]:
def getDiffs(np_arr, latest_val):
    
    col_max = np.amax(np_arr)
    col_min = np.amin(np_arr)
    diff_max = col_max - latest_val
    diff_min = col_min - latest_val
    
    return [col_max, col_min, diff_max, diff_min]

In [8]:
def create_data_window_features(df, window_size): #took thresh out

    pre_window = df.reset_index(drop=True)    
    window_rows = []

    
    for row_num in range(window_size, len(pre_window)):
        curr_window = pre_window.iloc[row_num-window_size:row_num]
        
        #Data modifications
        latest_vals = pre_window.iloc[row_num].tolist()
        window_means =  [curr_window["x"].mean(), curr_window["y"].mean(), curr_window["z"].mean(), curr_window["ax"].mean(), 
                            curr_window["ay"].mean(), curr_window["az"].mean(), curr_window["gsr"].mean(), curr_window["hr"].mean()]
        
        x_maxima_minima = getMaximaMinima(curr_window["x"])
        y_maxima_minima = getMaximaMinima(curr_window["y"])
        z_maxima_minima = getMaximaMinima(curr_window["z"])
        ax_maxima_minima = getMaximaMinima(curr_window["ax"])
        ay_maxima_minima = getMaximaMinima(curr_window["ay"])
        az_maxima_minima = getMaximaMinima(curr_window["az"])
        gsr_maxima_minima = getMaximaMinima(curr_window["gsr"])
        hr_maxima_minima = getMaximaMinima(curr_window["hr"])
        maxima_minimas = x_maxima_minima + y_maxima_minima + z_maxima_minima + ax_maxima_minima + ay_maxima_minima + az_maxima_minima + gsr_maxima_minima + hr_maxima_minima     
        
        x_diffs = getDiffs(curr_window["x"], latest_vals[0])
        y_diffs = getDiffs(curr_window["y"], latest_vals[1])
        z_diffs = getDiffs(curr_window["z"], latest_vals[2])
        ax_diffs = getDiffs(curr_window["ax"], latest_vals[3])
        ay_diffs = getDiffs(curr_window["ay"], latest_vals[4])
        az_diffs = getDiffs(curr_window["az"], latest_vals[5])
        gsr_diffs = getDiffs(curr_window["gsr"], latest_vals[6])
        hr_diffs = getDiffs(curr_window["hr"], latest_vals[7])
        all_diffs = x_diffs + y_diffs + z_diffs + ax_diffs + ay_diffs + az_diffs + gsr_diffs + hr_diffs 
        
        thresh = [latest_vals[6]- gsr_avg, latest_vals[7]- hr_avg]
        
        window_rows.append(latest_vals + window_means + maxima_minimas + all_diffs + thresh)
        
        
    window_df = pd.DataFrame(window_rows, columns = new_col_names)
  
        
    return window_df

In [9]:
def load_data(data_filenames, window_size, out_val): #took thresh out
    master_df = pd.DataFrame()
    i = 0

    for filename in data_filenames:

        load_data = pd.read_csv(filename, sep=",", names=column_names)
        load_data = load_data.drop('endline', 1).drop('timestamp', 1)

        master_df = master_df.append(create_data_window_features(load_data, window_size))
        i = i+1


    master_df['outcome'] = out_val   

    return master_df

In [10]:
wind_size = 20

master_a = load_data(data_a_files, wind_size, "A").tail(2200)
master_b = load_data(data_b_files, wind_size, "B")
master_c = load_data(data_c_files, wind_size, "C")
master_d = load_data(data_d_files, wind_size, "D")

In [11]:
print(len(master_a))
print(len(master_b))
print(len(master_c))
print(len(master_d))

2200
2209
1916
1492


In [12]:
###Combine all & shuffle
master_pre = master_a.append(master_b).append(master_c).append(master_d)
master_all = master_pre.sample(frac=1).reset_index(drop=True)

## Feature Preparation

In [13]:
#Split train/test/val
train, validate, test = np.split(master_all.sample(frac=1), [int(.6*len(master_all)), int(.8*len(master_all))])
print(len(train))
print(len(test))
print(len(validate))

4690
1564
1563


In [14]:
def outcome_encode(outcome):
    if outcome == "A":
        return [1.0, 0.0, 0.0, 0.0]
    elif outcome == "B":
        return [0.0, 1.0, 0.0, 0.0]
    elif outcome == "C":
        return [0.0, 0.0, 1.0, 0.0]
    else:
        return [0.0, 0.0, 0.0, 1.0]

In [15]:
###Split features/labels

train_l = [outcome_encode(row) for row in train[['outcome']].as_matrix().astype(str)]
train_f = train.drop('outcome', 1).as_matrix().astype(float)

test_l = [outcome_encode(row) for row in test[['outcome']].as_matrix().astype(str)]
test_f = test.drop('outcome', 1).as_matrix().astype(float)

val_l = [outcome_encode(row) for row in validate[['outcome']].as_matrix().astype(str)]
val_f = validate.drop('outcome', 1).as_matrix().astype(float)


## Tensorflow

In [16]:
# Parameters
learning_rate = 0.005
training_epochs = 40
batch_size = 300

# Network Parameters
n_hidden_1 = 100 # 1st layer
n_hidden_2 = 50 # 2nd layer
n_hidden_3 = 10 # 3rd layer
n_input = len(train_f[0])
n_classes = len(train_l[0])
n_rows = len(train_f) 

In [17]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])


# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}



# Create model func
def multilayer_perceptron(_X, _weights, _biases):
    
    #Hidden layer with RELU activation
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    
    #Hidden layer with sigmoid activation
    layer_2 = tf.sigmoid(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2'])) 
    
    #Hidden layer with sigmoid activation
    layer_3 = tf.sigmoid(tf.add(tf.matmul(layer_2, _weights['h3']), _biases['b3'])) 
    
    
    return (tf.matmul(layer_3, weights['out']) + biases['out'])


# Construct model
pred = multilayer_perceptron(x, weights, biases)



# Define loss and optimizer

# Softmax loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y)) 

# Adam Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) #RMSProp or Adam


# Initializing the variables
init = tf.global_variables_initializer()

## TF Session & Accuracy Check

In [18]:
# Savers for graph/variables
saver = tf.train.Saver()
saver_def = saver.as_saver_def()
print(saver_def.filename_tensor_name)
print(saver_def.restore_op_name)

save/Const:0
save/restore_all


In [21]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):

        avg_cost = 0.
        total_batch = int(float(n_rows)/float(batch_size))

        train_inputs = train_f
        train_labels = train_l

        # Loop over all batches
        for i in range(total_batch):

            start_ind = (i*batch_size)
            stop_ind = ((i+1)*batch_size)

            batch_xs = train_inputs[start_ind:stop_ind]
            batch_ys = train_labels[start_ind:stop_ind]


            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})

            # Compute average loss
            avg_cost += sess.run(cost, feed_dict={x: train_inputs, y: train_labels})/total_batch


            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
            print("\n---------------------------")

    print("Optimization Finished!")

    
    
    
######################################################################################################################

    print("Starting testing...") 

    test_inputs = test_f
    test_labels = test_l


#     Test model
    correct_prediction = tf.to_float(tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)))
    
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

    my_preds = (tf.cast(tf.argmax(pred, 1), "float")).eval({x: test_inputs, y: test_labels})
    my_labels = (tf.cast(tf.argmax(y, 1), "float")).eval({x: test_inputs, y: test_labels})   
    

    ###################################################################################################################
    
    # Validation dataset
    val_inputs = val_f
    val_labels = val_l


#     Test model
    correct_prediction = tf.to_float(tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)))
    
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

    val_preds = (tf.cast(tf.argmax(pred, 1), "float")).eval({x: val_inputs, y: val_labels})
    val_labels = (tf.cast(tf.argmax(y, 1), "float")).eval({x: val_inputs, y: val_labels})  
    
    
    print("Finished!")
    
    ######################################################################################################################

    
    # # write out 3 files
    saver.save(sess, os.path.join(os.getcwd(), 'trained_models/stress_model.sd'))
    tf.train.write_graph(sess.graph_def, '.', 'trained_models/stress_model.proto', as_text=False)
    tf.train.write_graph(sess.graph_def, '.', 'trained_models/stress_model.txt', as_text=True)


---------------------------

Epoch: 0001 cost= 0.138214413

---------------------------


---------------------------

Epoch: 0001 cost= 0.266395299

---------------------------


---------------------------

Epoch: 0001 cost= 0.386075703

---------------------------


---------------------------

Epoch: 0001 cost= 0.498474177

---------------------------


---------------------------

Epoch: 0001 cost= 0.604754384

---------------------------


---------------------------

Epoch: 0001 cost= 0.706237078

---------------------------


---------------------------

Epoch: 0001 cost= 0.804214724

---------------------------


---------------------------

Epoch: 0001 cost= 0.899532318

---------------------------


---------------------------

Epoch: 0001 cost= 0.992561007

---------------------------


---------------------------

Epoch: 0001 cost= 1.083579294

---------------------------


---------------------------

Epoch: 0001 cost= 1.172488014

---------------------------


---------

## Testing Model

In [22]:
print("Test size: ", my_preds.shape[0], "\n")

t_count = 0.0
f_count = 0.0

for i in range(my_preds.shape[0]):
    if my_preds[i] == my_labels[i]:
        t_count = t_count + 1
    elif my_preds[i] != my_labels[i]:
        f_count = f_count + 1
        
a_count = 0.0
b_count = 0.0
c_count = 0.0
d_count = 0.0

for i in range(my_labels.shape[0]):
    if my_labels[i] == 0.0:
        a_count = a_count + 1
    elif my_labels[i] == 1.0:
        b_count = b_count + 1
    elif my_labels[i] == 2.0:
        c_count = c_count + 1
    elif my_labels[i] == 3.0:
        d_count = d_count + 1
        
        
ta_count = 0.0
tb_count = 0.0
tc_count = 0.0
td_count = 0.0
f_count = 0.0

for i in range(my_preds.shape[0]):
    if my_preds[i] == my_labels[i] and my_labels[i] == 0.0:
        ta_count = ta_count + 1
    elif my_preds[i] == my_labels[i] and my_labels[i] == 1.0:
        tb_count = tb_count + 1
    elif my_preds[i] == my_labels[i] and my_labels[i] == 2.0:
        tc_count = tc_count + 1
    elif my_preds[i] == my_labels[i] and my_labels[i] == 3.0:
        td_count = td_count + 1
    elif my_preds[i] != my_labels[i]:
        f_count = f_count + 1
        
print("TA: ", '{0:.3f}'.format(float(ta_count)/float(a_count)), "\tPredicted correctly ", int(ta_count)," out of ", int(a_count), " As")
print("TB: ", '{0:.3f}'.format(float(tb_count)/float(b_count)), "\tPredicted correctly ", int(tb_count)," out of ", int(b_count), " Bs")
print("TC: ", '{0:.3f}'.format(float(tc_count)/float(c_count)), "\tPredicted correctly ", int(tc_count)," out of ", int(c_count), " Cs")
print("TD: ", '{0:.3f}'.format(float(td_count)/float(d_count)), "\tPredicted correctly ", int(td_count)," out of ", int(d_count), " Ds")

print("\nT: ", '{0:.3f}'.format(float(t_count)/float(my_preds.shape[0])), "% \tPredicted correctly ", int(t_count),
      " out of ", my_preds.shape[0] )
print("F: ", '{0:.3f}'.format(float(f_count)/float(my_preds.shape[0])), "% \tPredicted incorrectly ", int(f_count), 
      " out of ", my_preds.shape[0])

Test size:  1564 

TA:  0.980 	Predicted correctly  436  out of  445  As
TB:  0.888 	Predicted correctly  396  out of  446  Bs
TC:  0.894 	Predicted correctly  338  out of  378  Cs
TD:  0.905 	Predicted correctly  267  out of  295  Ds

T:  0.919 % 	Predicted correctly  1437  out of  1564
F:  0.081 % 	Predicted incorrectly  127  out of  1564


## Confusion Matrix 

In [23]:
num_a_as_a = 0.0
num_a_as_b = 0.0
num_a_as_c = 0.0
num_a_as_d = 0.0
num_b_as_a = 0.0
num_b_as_b = 0.0
num_b_as_c = 0.0
num_b_as_d = 0.0
num_c_as_a = 0.0
num_c_as_b = 0.0
num_c_as_c = 0.0
num_c_as_d = 0.0
num_d_as_a = 0.0
num_d_as_b = 0.0
num_d_as_c = 0.0
num_d_as_d = 0.0

for i in range(my_preds.shape[0]):
    if my_preds[i] == my_labels[i]:
        if my_labels[i] == 0.0:
            num_a_as_a += 1.0
        elif my_labels[i] == 1.0:
            num_b_as_b += 1.0
        elif my_labels[i] == 2.0:
            num_c_as_c += 1.0
        elif my_labels[i] == 3.0:
            num_d_as_d += 1.0
    else:
        if my_labels[i] == 0.0 and my_preds[i] == 1.0:
            num_a_as_b += 1.0
        elif my_labels[i] == 0.0 and my_preds[i] == 2.0:
            num_a_as_c += 1.0
        elif my_labels[i] == 0.0 and my_preds[i] == 3.0:
            num_a_as_d += 1.0
            
        elif my_labels[i] == 1.0 and my_preds[i] == 0.0:
            num_b_as_a += 1.0
        elif my_labels[i] == 1.0 and my_preds[i] == 2.0:
            num_b_as_c += 1.0
        elif my_labels[i] == 1.0 and my_preds[i] == 3.0:
            num_b_as_d += 1.0
            
        elif my_labels[i] == 2.0 and my_preds[i] == 0.0:
            num_c_as_a += 1.0
        elif my_labels[i] == 2.0 and my_preds[i] == 1.0:
            num_c_as_b += 1.0
        elif my_labels[i] == 2.0 and my_preds[i] == 3.0:
            num_c_as_d += 1.0
            
        elif my_labels[i] == 3.0 and my_preds[i] == 0.0:
            num_d_as_a += 1.0
        elif my_labels[i] == 3.0 and my_preds[i] == 1.0:
            num_d_as_b += 1.0
        elif my_labels[i] == 3.0 and my_preds[i] == 2.0:
            num_d_as_c += 1.0
        

In [24]:
print("Label\t\t\tPrediction")
print("-------------------------------------")
print("\t|\t__A__\t__B__\t__C__\t__D__")
print("A\t|\t", num_a_as_a, "\t", num_a_as_b, "\t", num_a_as_c, "\t", num_a_as_d)
print("B\t|\t", num_b_as_a, "\t", num_b_as_b, "\t", num_b_as_c, "\t", num_b_as_d)
print("C\t|\t", num_c_as_a, "\t", num_c_as_b, "\t", num_c_as_c, "\t", num_c_as_d)
print("D\t|\t", num_d_as_a, "\t", num_d_as_b, "\t", num_d_as_c, "\t", num_d_as_d)

Label			Prediction
-------------------------------------
	|	__A__	__B__	__C__	__D__
A	|	 436.0 	 9.0 	 0.0 	 0.0
B	|	 49.0 	 396.0 	 0.0 	 1.0
C	|	 0.0 	 4.0 	 338.0 	 36.0
D	|	 0.0 	 1.0 	 27.0 	 267.0


## Validation dataset

In [25]:
print("Test size: ", val_preds.shape[0], "\n")

t_count = 0.0
f_count = 0.0

for i in range(val_preds.shape[0]):
    if val_preds[i] == val_labels[i]:
        t_count = t_count + 1
    elif val_preds[i] != val_labels[i]:
        f_count = f_count + 1
        
a_count = 0.0
b_count = 0.0
c_count = 0.0
d_count = 0.0

for i in range(val_labels.shape[0]):
    if val_labels[i] == 0.0:
        a_count = a_count + 1
    elif val_labels[i] == 1.0:
        b_count = b_count + 1
    elif val_labels[i] == 2.0:
        c_count = c_count + 1
    elif val_labels[i] == 3.0:
        d_count = d_count + 1
        
        
ta_count = 0.0
tb_count = 0.0
tc_count = 0.0
td_count = 0.0
f_count = 0.0

for i in range(val_preds.shape[0]):
    if val_preds[i] == val_labels[i] and val_labels[i] == 0.0:
        ta_count = ta_count + 1
    elif val_preds[i] == val_labels[i] and val_labels[i] == 1.0:
        tb_count = tb_count + 1
    elif val_preds[i] == val_labels[i] and val_labels[i] == 2.0:
        tc_count = tc_count + 1
    elif val_preds[i] == val_labels[i] and val_labels[i] == 3.0:
        td_count = td_count + 1
    elif val_preds[i] != val_labels[i]:
        f_count = f_count + 1
        
print("TA: ", '{0:.3f}'.format(float(ta_count)/float(a_count)), "\tPredicted correctly ", int(ta_count)," out of ", int(a_count), " As")
print("TB: ", '{0:.3f}'.format(float(tb_count)/float(b_count)), "\tPredicted correctly ", int(tb_count)," out of ", int(b_count), " Bs")
print("TC: ", '{0:.3f}'.format(float(tc_count)/float(c_count)), "\tPredicted correctly ", int(tc_count)," out of ", int(c_count), " Cs")
print("TD: ", '{0:.3f}'.format(float(td_count)/float(d_count)), "\tPredicted correctly ", int(td_count)," out of ", int(d_count), " Ds")

print("\nT: ", '{0:.3f}'.format(float(t_count)/float(val_preds.shape[0])), "% \tPredicted correctly ", int(t_count),
      " out of ", val_preds.shape[0] )
print("F: ", '{0:.3f}'.format(float(f_count)/float(val_preds.shape[0])), "% \tPredicted incorrectly ", int(f_count), 
      " out of ", val_preds.shape[0])

Test size:  1563 

TA:  0.959 	Predicted correctly  397  out of  414  As
TB:  0.892 	Predicted correctly  415  out of  465  Bs
TC:  0.904 	Predicted correctly  347  out of  384  Cs
TD:  0.957 	Predicted correctly  287  out of  300  Ds

T:  0.925 % 	Predicted correctly  1446  out of  1563
F:  0.075 % 	Predicted incorrectly  117  out of  1563


In [26]:
num_a_as_a = 0.0
num_a_as_b = 0.0
num_a_as_c = 0.0
num_a_as_d = 0.0
num_b_as_a = 0.0
num_b_as_b = 0.0
num_b_as_c = 0.0
num_b_as_d = 0.0
num_c_as_a = 0.0
num_c_as_b = 0.0
num_c_as_c = 0.0
num_c_as_d = 0.0
num_d_as_a = 0.0
num_d_as_b = 0.0
num_d_as_c = 0.0
num_d_as_d = 0.0

for i in range(val_preds.shape[0]):
    if val_preds[i] == val_labels[i]:
        if val_labels[i] == 0.0:
            num_a_as_a += 1.0
        elif val_labels[i] == 1.0:
            num_b_as_b += 1.0
        elif val_labels[i] == 2.0:
            num_c_as_c += 1.0
        elif val_labels[i] == 3.0:
            num_d_as_d += 1.0
    else:
        if val_labels[i] == 0.0 and val_preds[i] == 1.0:
            num_a_as_b += 1.0
        elif val_labels[i] == 0.0 and val_preds[i] == 2.0:
            num_a_as_c += 1.0
        elif val_labels[i] == 0.0 and val_preds[i] == 3.0:
            num_a_as_d += 1.0
            
        elif val_labels[i] == 1.0 and val_preds[i] == 0.0:
            num_b_as_a += 1.0
        elif val_labels[i] == 1.0 and val_preds[i] == 2.0:
            num_b_as_c += 1.0
        elif val_labels[i] == 1.0 and val_preds[i] == 3.0:
            num_b_as_d += 1.0
            
        elif val_labels[i] == 2.0 and val_preds[i] == 0.0:
            num_c_as_a += 1.0
        elif val_labels[i] == 2.0 and val_preds[i] == 1.0:
            num_c_as_b += 1.0
        elif val_labels[i] == 2.0 and val_preds[i] == 3.0:
            num_c_as_d += 1.0
            
        elif val_labels[i] == 3.0 and val_preds[i] == 0.0:
            num_d_as_a += 1.0
        elif val_labels[i] == 3.0 and val_preds[i] == 1.0:
            num_d_as_b += 1.0
        elif val_labels[i] == 3.0 and val_preds[i] == 2.0:
            num_d_as_c += 1.0
        

In [27]:
print("Label\t\t\tPrediction")
print("-------------------------------------")
print("\t|\t__A__\t__B__\t__C__\t__D__")
print("A\t|\t", num_a_as_a, "\t", num_a_as_b, "\t", num_a_as_c, "\t", num_a_as_d)
print("B\t|\t", num_b_as_a, "\t", num_b_as_b, "\t", num_b_as_c, "\t", num_b_as_d)
print("C\t|\t", num_c_as_a, "\t", num_c_as_b, "\t", num_c_as_c, "\t", num_c_as_d)
print("D\t|\t", num_d_as_a, "\t", num_d_as_b, "\t", num_d_as_c, "\t", num_d_as_d)

Label			Prediction
-------------------------------------
	|	__A__	__B__	__C__	__D__
A	|	 397.0 	 15.0 	 2.0 	 0.0
B	|	 45.0 	 415.0 	 2.0 	 3.0
C	|	 1.0 	 1.0 	 347.0 	 35.0
D	|	 0.0 	 0.0 	 13.0 	 287.0
